# AI Flappy Bird Analysis

## Flappy Bird no-GUI game engine

### Game-specific imports and constants

In [22]:
import random
import time
import json
import csv
from google.colab import files
import datetime
from zoneinfo import ZoneInfo
import zipfile

GAME_WIDTH = 500
GAME_HEIGHT = 700
MAX_EPISODES = 3
FIXED_DT = 0.0167

### Classes

#### Bird class

In [23]:
class Bird:
  def __init__(self, game):
    self.game = game

    self.isFlapping = False

    self.width = self.game.BIRD_WIDTH
    self.height = self.game.BIRD_HEIGHT

    self.x = self.game.GAME_WIDTH * self.game.BIRD_X_RATIO
    self._y = (self.game.GAME_HEIGHT - self.height) / 2

    self.velocity = 0

  @property
  def y(self):
    return self._y;
  @y.setter
  def y(self, v):
    if (v > self.game.GAME_HEIGHT or v < 0 - self.height):
      self.game.gameIsOver = True
    else:
     self._y = v;

  @property
  def hitbox(self):
    return {
      'x': self.x,
      'y': self.y,
      'width': self.width,
      'height': self.height,
    }

  def update(self, dt):
    self.fall(dt)

  def reset(self):
    self.x = self.game.GAME_WIDTH * self.game.BIRD_X_RATIO
    self._y = (self.game.GAME_HEIGHT - self.height) / 2

    self.velocity = 0

  def fall(self, dt):
    self.velocity += self.game.GRAVITY * dt
    self.y += self.velocity * dt

  def flapWings(self):
    self.velocity = -300


#### Pipes class

In [24]:
class Pipes:
  def __init__(self, game):
    self.game = game

    self.startPoint = 500

    self.gap = self.game.PIPE_GAP

    self._x = self.game.PIPE_START_POINT
    self.topY = random.random() * (self.game.GAME_HEIGHT - self.gap)
    self.botY = self.topY + self.gap

    self.width = self.game.PIPE_WIDTH
    self.topHeight = self.topY
    self.botHeight = self.game.GAME_HEIGHT - self.botY

    self.velocity = self.game.PIPE_VELOCITY

    self.passed = False

  @property
  def x(self):
    return self._x
  @x.setter
  def x(self, v):
    if (v + self.width < 0):
      self.game.removePipePair(self)
    else:
      self._x = v

  @property
  def topHitbox(self):
    return {
      'x': self.x,
      'y': 0,
      'width': self.width,
      'height': self.topHeight,
    }
  @property
  def botHitbox(self):
    return {
      'x': self.x,
      'y': self.botY,
      'width': self.width,
      'height': self.botHeight,
    }

  def update(self, dt):
    self.x -= self.velocity * dt

  def reset(self):
    self.x = self.game.PIPE_START_POINT
    self.topY = random.random() * (self.game.GAME_HEIGHT - self.gap)
    self.botY = self.topY + self.gap

    self.topHeight = self.topY
    self.botHeight = self.game.GAME_HEIGHT - self.botY


#### Game class

In [25]:
class Game:
  def __init__(self, GAME_WIDTH, GAME_HEIGHT):
    self.frameStates = []
    self.episodeId = 0
    self.frameId = 0

    self.AGENT_TYPE = "human"
    self.GAME_WIDTH = GAME_WIDTH
    self.GAME_HEIGHT = GAME_HEIGHT
    self.BIRD_X_RATIO = 0.3
    self.BIRD_WIDTH = 50
    self.BIRD_HEIGHT = 40
    self.GRAVITY = 600
    self.PIPE_START_POINT = 500
    self.PIPE_GAP = 200
    self.PIPE_WIDTH = 75
    self.PIPE_VELOCITY = 100
    self.PIPES_INTERVAL = 3

    self.pipesElapsed = 0

    self.bird = Bird(self)
    self.pipes = []

    self.gameIsOver = False

    self.points = 0

    self.metadata = {
      "agent_type": self.AGENT_TYPE,
      "game_width": self.GAME_WIDTH,
      "game_height": self.GAME_HEIGHT,
      "bird_x_ratio": self.BIRD_X_RATIO,
      "bird_width": self.BIRD_WIDTH,
      "bird_height": self.BIRD_HEIGHT,
      "gravity": self.GRAVITY,
      "pipe_gap": self.PIPE_GAP,
      "pipe_width": self.PIPE_WIDTH,
      "pipe_velocity": self.PIPE_VELOCITY,
      "pipes_interval": self.PIPES_INTERVAL,
    }

  def update(self, dt):
    for pipePair in self.pipes:
      if self.areColliding(self.bird.hitbox, pipePair.topHitbox) or \
      self.areColliding(self.bird.hitbox, pipePair.botHitbox):
        self.gameIsOver = True
        return

      if not(pipePair.passed) and self.bird.x > pipePair.x:
        self.points += 1
        pipePair.passed = True

      pipePair.update(dt)
    self.bird.update(dt)

  def gameLoop(self, dt):
    if not self.gameIsOver:
      self.update(dt)

    if self.gameIsOver:
      if self.episodeId + 1 >= MAX_EPISODES:
        return

      self.gameIsOver = False
      self.reset()
    else:
      self.frameStates.append({
        'episode_id': self.episodeId,
        'frame_id': self.frameId,
        'bird_y': self.bird.y,
        'bird_velocity': self.bird.velocity,
        'action': 1 if self.bird.isFlapping else 0,
        'pipe_pair_number': len(self.pipes),
        'pipes': json.dumps([
          {
            "x": pipe_pair.x,
            "top_y": pipe_pair.topY,
            "bot_y": pipe_pair.botY,
            "passed": pipe_pair.passed
          }
          for pipe_pair in self.pipes
        ]) if len(self.pipes) > 0 else 'null',
        'score': self.points,
        'game_is_over': 1 if self.gameIsOver else 0,
      })

      self.pipesElapsed += dt

      if self.pipesElapsed >= self.PIPES_INTERVAL:
        self.addPipePair()
        self.pipesElapsed = 0

      self.bird.isFlapping = False;
      self.frameId += 1

  def reset(self):
    self.episodeId += 1
    self.frameId = 0

    self.pipesElapsed = 0

    self.bird.reset()

    self.pipes = []

    self.points = 0

  def addPipePair(self):
    pipePair = Pipes(self)
    self.pipes.append(pipePair)
  def removePipePair(self, pipePair):
    if pipePair in self.pipes:
      self.pipes.remove(pipePair)

  def areColliding(self, bird, pipe):
    return bird['x'] < pipe['x'] + pipe['width'] and \
      bird['x'] + bird['width'] > pipe['x'] and \
      bird['y'] < pipe['y'] + pipe['height'] and \
      bird['y'] + bird['height'] > pipe['y'] - 30


### Start and export game

In [27]:
game = Game(GAME_WIDTH, GAME_HEIGHT)

while (not game.gameIsOver) or game.episodeId + 1 < MAX_EPISODES:
  game.gameLoop(FIXED_DT)

frames_headers = list(game.frameStates[0].keys())

frames_csv = "frames.csv"
with open(frames_csv, "w", newline="", encoding="utf-8") as f:
  writer = csv.writer(f)
  writer.writerow(frames_headers)
  writer.writerows(
    [[frame[h] for h in frames_headers] for frame in game.frameStates]
  )

metadata_csv = "metadata.csv"

meta_keys = list(game.metadata.keys())
meta_values = list(game.metadata.values())

with open(metadata_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(meta_keys)
    writer.writerow(meta_values)

now = datetime.datetime.now(ZoneInfo("Europe/Zagreb"))

pad = lambda n: str(n).zfill(2)

current_date = f"{now.year}{pad(now.month)}{pad(now.day)}"
current_time = f"{pad(now.hour)}{pad(now.minute)}{pad(now.second)}"

timestamp = now.strftime("%Y%m%d_%H%M%S")

zip_filename = f"flappy_bird_game_{timestamp}.zip"

with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
  zipf.write(frames_csv)
  zipf.write(metadata_csv)

files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>